In [15]:
%load_ext autoreload
%autoreload 2

In [17]:
from state import get_state, queue
from reward import calculate_reward
from learner import Learner
from traffic_lights_maps import get_traffic_light_phases, make_map
import os
import sys

import traci
import random
import gym
from gym import spaces

In [ ]:
# if traci.isLoaded():
#     traci.close()

In [27]:
sumoBinary = r"C:/Program Files/rl_project/Eclipse/Sumo/bin/sumo-gui.exe" 
sumoConfig = r"C:/Program Files/rl_project/2025-02-12-16-24-37/osm.sumocfg"
sumoConfig2 = r"C:/Program Files/rl_project/2025-02-25-19-05-44/osm.sumocfg"
sumoConfig3 = r"C:/Program Files/rl_project/Traditional_traffic/traditional_traffic.sumo.cfg"


sumoCmd = [sumoBinary, "-c", sumoConfig3, "--start"]


traci.start(sumoCmd)

trafic_light_ids = traci.trafficlight.getIDList()
print("Les feux:", trafic_light_ids)

detector_ids = traci.inductionloop.getIDList()
print("Détecteurs:", detector_ids)

if not trafic_light_ids:
    print("No traffic lights, issues")
    traci.close()
    sys.exit()


lane_ids = traci.lane.getIDList()
print("lane id", lane_ids, len(lane_ids))


actions_map = make_map(trafic_light_ids)
print("Carte des actions :", actions_map)
print(f"nombre de feux : {len(trafic_light_ids)}")

action_space = spaces.Discrete(len(actions_map))  # Nombre d'actions = nombre de phases
print("Espace d'actions :", action_space)


# l'agent
state_space_size = traci.inductionloop.getIDCount() * 2
agent = Learner(state_space_size, action_space, 0.0)


# Ini
state = get_state(detector_ids)
total_reward = 0
step = 0
lane_situation = queue(lane_ids)

print ("###")
for traffic_light_id in trafic_light_ids:
    print(traci.trafficlight.getAllProgramLogics(traffic_light_id))

print('###')

print(f"nombre de détecteurs {len(detector_ids)}")
print(f"longueur de state {len(state)}")
print("Simu")

try:
    while step < 100000:
        action = agent.act(state)

        if action not in actions_map:
            print(f"action {action} not in action_map")
            break

        lights_phase = actions_map[action]

        # changement phases des feux
        for light, phase in zip(trafic_light_ids, lights_phase):
            traci.trafficlight.setPhase(light, phase)

        traci.simulationStep()

        # Synthèse
        # print(f"Étape {step}, Action {action}, Feux {lights_phase},")
        print(f"Etat {get_state(detector_ids)}")
        print(f"occupation lane {queue(lane_ids)}")
        # update of the situation
        next_state = get_state(detector_ids)
        reward = calculate_reward(state, next_state)
        total_reward += reward
        state = next_state
        step += 1




except Exception as e:
    print(f"exception with: {e}")

finally:
    traci.close()
    print("end")

Les feux: ('junction',)
Détecteurs: ()
lane id (':junction_0_0', ':junction_10_0', ':junction_11_0', ':junction_1_0', ':junction_2_0', ':junction_3_0', ':junction_4_0', ':junction_5_0', ':junction_6_0', ':junction_7_0', ':junction_8_0', ':junction_9_0', 'end1_junction_0', 'end1_junction_1', 'end1_junction_2', 'end2_junction_0', 'end2_junction_1', 'end2_junction_2', 'end3_junction_0', 'end3_junction_1', 'end3_junction_2', 'end4_junction_0', 'end4_junction_1', 'end4_junction_2', 'junction_end1_0', 'junction_end1_1', 'junction_end1_2', 'junction_end2_0', 'junction_end2_1', 'junction_end2_2', 'junction_end3_0', 'junction_end3_1', 'junction_end3_2', 'junction_end4_0', 'junction_end4_1', 'junction_end4_2') 36
Carte des actions : {0: [0, 1, 2, 3, 4, 5, 6, 7]}
nombre de feux : 1
Espace d'actions : Discrete(1)
###
(Logic(programID='0', type=0, currentPhaseIndex=0, phases=(Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=5.0, state='gyrgrrgyrgrr', minDur=5.0, 